# Regression Analysis

Regression analysis models the relationship between a **response variable** (dependent) and one or more **predictor variables** (independent). It is the workhorse of statistical modelling and machine learning.

**Topics covered in this notebook:**
1. Simple linear regression — fitting a line, OLS estimation
2. Interpreting coefficients — R², p-values, residual diagnostics
3. Multiple linear regression — multiple predictors
4. Polynomial regression — non-linear patterns, bias-variance tradeoff
5. Logistic regression — binary outcomes, sigmoid function
6. Regression assumptions — linearity, normality, homoscedasticity

**Mathematical foundation — Ordinary Least Squares (OLS):**

$$\hat{\boldsymbol{\beta}} = (\mathbf{X}^\top \mathbf{X})^{-1} \mathbf{X}^\top \mathbf{y}$$

OLS minimises the **residual sum of squares (RSS)** $\sum_{i=1}^{n}(y_i - \hat{y}_i)^2$ and is the **BLUE** (Best Linear Unbiased Estimator) under the Gauss-Markov assumptions.

In [ ]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Optional: scikit-learn (used where noted; gracefully falls back to manual)
try:
    from sklearn.linear_model import LinearRegression, LogisticRegression
    from sklearn.preprocessing import PolynomialFeatures, StandardScaler
    from sklearn.pipeline import make_pipeline
    from sklearn.metrics import r2_score
    SKLEARN = True
    print('scikit-learn is available — using sklearn where convenient.')
except ImportError:
    SKLEARN = False
    print('scikit-learn not found — using NumPy/SciPy only.')

rng = np.random.default_rng(seed=0)

plt.rcParams.update({
    'figure.dpi': 100,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
})

print('NumPy:', np.__version__)
print('SciPy:', __import__('scipy').__version__)

## 1. Simple Linear Regression

The simple linear model relates a single predictor $x$ to the response $y$:

$$y_i = \beta_0 + \beta_1 x_i + \varepsilon_i, \qquad \varepsilon_i \overset{\text{i.i.d.}}{\sim} \mathcal{N}(0, \sigma^2)$$

OLS closed-form estimates:

$$\hat{\beta}_1 = \frac{\sum(x_i - \bar{x})(y_i - \bar{y})}{\sum(x_i - \bar{x})^2} = \frac{S_{xy}}{S_{xx}}, \qquad \hat{\beta}_0 = \bar{y} - \hat{\beta}_1 \bar{x}$$

The **95% confidence band** around the regression line reflects uncertainty in the estimated mean response $E[y \mid x]$. The wider **prediction interval** additionally accounts for individual observation noise $\sigma^2$.

In [ ]:
# Scenario: predict house price (€ thousands) from size (m²)
n = 80
x = rng.uniform(40, 200, size=n)                            # house size (m²)
true_beta0, true_beta1 = 30.0, 2.5                         # true intercept, slope
noise_std = 25.0
y = true_beta0 + true_beta1 * x + rng.normal(0, noise_std, size=n)

# OLS via scipy.stats.linregress
slope, intercept, r_value, p_value, stderr = stats.linregress(x, y)

print(f'Fitted intercept β₀ : {intercept:.4f}  (true: {true_beta0})')
print(f'Fitted slope     β₁ : {slope:.4f}  (true: {true_beta1})')
print(f'R                   : {r_value:.4f}')
print(f'R²                  : {r_value**2:.4f}')
print(f'p-value (slope)     : {p_value:.6f}')
print(f'Std error (slope)   : {stderr:.4f}')

# Predictions + 95% confidence band (analytical formula)
x_grid = np.linspace(x.min(), x.max(), 300)
y_hat  = intercept + slope * x_grid

x_mean = x.mean()
n_obs  = len(x)
Sxx = ((x - x_mean) ** 2).sum()
residuals = y - (intercept + slope * x)
s_e = np.sqrt((residuals ** 2).sum() / (n_obs - 2))       # residual std error

t_crit = stats.t.ppf(0.975, df=n_obs - 2)
se_mean = s_e * np.sqrt(1 / n_obs + (x_grid - x_mean) ** 2 / Sxx)
se_pred = s_e * np.sqrt(1 + 1 / n_obs + (x_grid - x_mean) ** 2 / Sxx)

ci_lower = y_hat - t_crit * se_mean
ci_upper = y_hat + t_crit * se_mean
pi_lower = y_hat - t_crit * se_pred
pi_upper = y_hat + t_crit * se_pred

# Plot
fig, ax = plt.subplots(figsize=(8, 5))
ax.scatter(x, y, color='steelblue', alpha=0.6, s=30, label='Observations')
ax.plot(x_grid, y_hat, 'r-', lw=2, label=f'OLS fit: y = {intercept:.1f} + {slope:.2f}x')
ax.fill_between(x_grid, ci_lower, ci_upper, color='red', alpha=0.15,
                label='95% confidence band')
ax.fill_between(x_grid, pi_lower, pi_upper, color='orange', alpha=0.10,
                label='95% prediction interval')
ax.set_xlabel('House size (m²)')
ax.set_ylabel('Price (€ thousands)')
ax.set_title(f'Simple Linear Regression  |  R²={r_value**2:.3f}, p={p_value:.4f}')
ax.legend()
plt.tight_layout()
plt.show()

## 2. Interpreting Coefficients: R², Residuals, and Q-Q Plot

**R² (coefficient of determination)** measures the proportion of variance in $y$ explained by the model:

$$R^2 = 1 - \frac{\text{RSS}}{\text{TSS}} = 1 - \frac{\sum(y_i - \hat{y}_i)^2}{\sum(y_i - \bar{y})^2}$$

**Residual diagnostics** are essential for validating OLS assumptions:

| Plot | What to look for | Violation suggests |
|------|------------------|-----------------------|
| Residuals vs Fitted | Random scatter around 0 | Non-linearity or heteroscedasticity |
| Q-Q plot | Points on diagonal | Non-normality of errors |
| Scale-Location | Horizontal band | Heteroscedasticity |
| Residuals vs Leverage | No high-leverage outliers | Influential observations |

The **p-value of the slope** tests H₀: β₁ = 0 (no linear relationship). Reject when p < α.

In [ ]:
# Re-use the house price data from Cell 4
y_fitted   = intercept + slope * x
residuals  = y - y_fitted
std_resid  = residuals / s_e                               # standardised residuals

RSS = (residuals ** 2).sum()
TSS = ((y - y.mean()) ** 2).sum()
R2  = 1 - RSS / TSS
adj_R2 = 1 - (1 - R2) * (n_obs - 1) / (n_obs - 2)

print(f'R²          : {R2:.4f}')
print(f'Adjusted R² : {adj_R2:.4f}')
print(f'RSS         : {RSS:.2f}')
print(f'Residual SE : {s_e:.4f}')

# Shapiro-Wilk test on residuals
sw_stat, sw_p = stats.shapiro(residuals)
print(f'\nShapiro-Wilk test on residuals: W={sw_stat:.4f}, p={sw_p:.4f}')
print('Residuals appear normal.' if sw_p > 0.05 else 'Residuals may NOT be normal.')

fig = plt.figure(figsize=(11, 8))
gs  = gridspec.GridSpec(2, 2, hspace=0.40, wspace=0.35)

# 1. Residuals vs Fitted
ax1 = fig.add_subplot(gs[0, 0])
ax1.scatter(y_fitted, residuals, alpha=0.6, color='steelblue', s=25)
ax1.axhline(0, color='red', lw=1.5, ls='--')
ax1.set_xlabel('Fitted values'); ax1.set_ylabel('Residuals')
ax1.set_title('Residuals vs Fitted')

# 2. Q-Q plot
ax2 = fig.add_subplot(gs[0, 1])
(osm, osr), (slope_qq, intercept_qq, r_qq) = stats.probplot(residuals, dist='norm')
ax2.scatter(osm, osr, alpha=0.6, color='steelblue', s=25)
ax2.plot(osm, slope_qq * np.array(osm) + intercept_qq, 'r-', lw=1.5)
ax2.set_xlabel('Theoretical quantiles'); ax2.set_ylabel('Sample quantiles')
ax2.set_title('Normal Q-Q Plot')

# 3. Scale-Location (sqrt |standardised residuals| vs fitted)
ax3 = fig.add_subplot(gs[1, 0])
ax3.scatter(y_fitted, np.sqrt(np.abs(std_resid)), alpha=0.6, color='steelblue', s=25)
ax3.set_xlabel('Fitted values'); ax3.set_ylabel('√|Standardised residuals|')
ax3.set_title('Scale-Location')

# 4. Residual histogram
ax4 = fig.add_subplot(gs[1, 1])
ax4.hist(residuals, bins=14, color='steelblue', edgecolor='white', alpha=0.8, density=True)
xr = np.linspace(residuals.min(), residuals.max(), 200)
ax4.plot(xr, stats.norm.pdf(xr, residuals.mean(), residuals.std()), 'r-', lw=2)
ax4.set_xlabel('Residual'); ax4.set_ylabel('Density')
ax4.set_title('Residual Distribution')

fig.suptitle(f'Residual Diagnostics  |  R²={R2:.3f}', fontsize=13, y=1.01)
plt.show()

## 3. Multiple Linear Regression

With $p$ predictors the model becomes:

$$\mathbf{y} = \mathbf{X}\boldsymbol{\beta} + \boldsymbol{\varepsilon}, \qquad \hat{\boldsymbol{\beta}} = (\mathbf{X}^\top\mathbf{X})^{-1}\mathbf{X}^\top\mathbf{y}$$

where $\mathbf{X}$ is the $n \times (p+1)$ design matrix (first column all ones for the intercept).

**Key considerations:**
- **Multicollinearity**: when predictors are correlated, coefficient estimates become unstable. Check with Variance Inflation Factor (VIF).
- **Adjusted R²**: penalises adding uninformative predictors — always report alongside R².
- **F-statistic**: tests whether *all* coefficients (except intercept) are simultaneously zero.

$$F = \frac{(\text{TSS} - \text{RSS})/p}{\text{RSS}/(n - p - 1)}$$

In [ ]:
# Scenario: predict salary (€k) from years of experience, education level (0-3), and performance score
n = 120
experience  = rng.uniform(0, 30, n)
education   = rng.integers(0, 4, n).astype(float)
performance = rng.uniform(50, 100, n)

# True model: salary = 25 + 1.8*exp + 5*edu + 0.3*perf + noise
true_betas = np.array([25.0, 1.8, 5.0, 0.3])
noise = rng.normal(0, 8, n)
y = true_betas[0] + true_betas[1]*experience + true_betas[2]*education + true_betas[3]*performance + noise

# Build design matrix (with intercept column)
X_raw = np.column_stack([experience, education, performance])
ones  = np.ones((n, 1))
X     = np.hstack([ones, X_raw])                           # shape: (n, 4)

# OLS via normal equations
beta_hat = np.linalg.lstsq(X, y, rcond=None)[0]

y_hat    = X @ beta_hat
residuals = y - y_hat
p_pred   = X.shape[1] - 1                                  # number of predictors (excl. intercept)
RSS      = (residuals ** 2).sum()
TSS      = ((y - y.mean()) ** 2).sum()
R2       = 1 - RSS / TSS
adj_R2   = 1 - (1 - R2) * (n - 1) / (n - p_pred - 1)
s2       = RSS / (n - p_pred - 1)

# Standard errors of coefficients
XtX_inv  = np.linalg.inv(X.T @ X)
se_betas = np.sqrt(np.diag(XtX_inv) * s2)

# t-statistics and p-values
t_stats = beta_hat / se_betas
df_res  = n - p_pred - 1
p_vals  = 2 * stats.t.sf(np.abs(t_stats), df=df_res)

# F-statistic for overall model significance
F_stat = ((TSS - RSS) / p_pred) / (RSS / df_res)
F_pval = stats.f.sf(F_stat, p_pred, df_res)

print('Multiple Linear Regression Results')
print('=' * 55)
print(f'R²: {R2:.4f}   Adjusted R²: {adj_R2:.4f}')
print(f'F-statistic: {F_stat:.2f}  (p={F_pval:.2e})')
print(f'n={n}, p={p_pred}')
print()
labels = ['Intercept', 'Experience', 'Education', 'Performance']
print(f'{"Variable":<14} {"Estimate":>10} {"Std Err":>10} {"t-stat":>10} {"p-value":>12}')
print('-' * 58)
for lbl, b, se, t, p in zip(labels, beta_hat, se_betas, t_stats, p_vals):
    sig = '***' if p < 0.001 else ('**' if p < 0.01 else ('*' if p < 0.05 else ''))
    print(f'{lbl:<14} {b:>10.4f} {se:>10.4f} {t:>10.4f} {p:>12.4f} {sig}')

# Coefficient plot
fig, ax = plt.subplots(figsize=(7, 4))
ci_mult = stats.t.ppf(0.975, df=df_res)
ci_half = ci_mult * se_betas[1:]                           # skip intercept
y_pos = np.arange(p_pred)
ax.barh(y_pos, beta_hat[1:], xerr=ci_half, color='steelblue', alpha=0.7,
        edgecolor='white', capsize=4)
ax.axvline(0, color='black', lw=1)
ax.set_yticks(y_pos)
ax.set_yticklabels(['Experience', 'Education', 'Performance'])
ax.set_xlabel('Coefficient estimate (with 95% CI)')
ax.set_title('Multiple Regression Coefficients')
plt.tight_layout()
plt.show()

## 4. Polynomial Regression

When the relationship between $x$ and $y$ is non-linear, we can extend OLS by including polynomial features:

$$y = \beta_0 + \beta_1 x + \beta_2 x^2 + \cdots + \beta_d x^d + \varepsilon$$

This is still *linear* in the parameters $\boldsymbol{\beta}$ — just a linear model in a transformed feature space.

**Bias-variance tradeoff:**
- Low-degree polynomial → high bias (underfitting)
- High-degree polynomial → high variance (overfitting)
- The optimal degree minimises prediction error on unseen data (use cross-validation)

The training R² always increases with degree, so use **adjusted R²** or **AIC/BIC** to compare models.

In [ ]:
# Scenario: model a noisy sinusoidal relationship
n = 60
x_poly = rng.uniform(-3, 3, size=n)
y_poly = np.sin(x_poly) + rng.normal(0, 0.3, size=n)

x_grid_poly = np.linspace(-3.2, 3.2, 300)
degrees = [1, 3, 7, 15]

fig, axes = plt.subplots(2, 2, figsize=(11, 8), sharey=True)
axes = axes.ravel()

r2_scores = []
for ax, deg in zip(axes, degrees):
    # Build Vandermonde matrix for training data and grid
    X_train = np.vander(x_poly,      N=deg + 1, increasing=True)
    X_grid  = np.vander(x_grid_poly, N=deg + 1, increasing=True)

    coeffs = np.linalg.lstsq(X_train, y_poly, rcond=None)[0]
    y_pred_train = X_train @ coeffs
    y_pred_grid  = X_grid  @ coeffs

    rss = ((y_poly - y_pred_train) ** 2).sum()
    tss = ((y_poly - y_poly.mean()) ** 2).sum()
    r2  = 1 - rss / tss
    adj_r2 = 1 - (1 - r2) * (n - 1) / (n - deg - 1)
    r2_scores.append(r2)

    ax.scatter(x_poly, y_poly, color='steelblue', alpha=0.5, s=20, label='Data')
    ax.plot(x_grid_poly, np.sin(x_grid_poly), 'g--', lw=1.5, label='True sin(x)')
    ax.plot(x_grid_poly, y_pred_grid, 'r-', lw=2,
            label=f'Degree {deg}')
    ax.set_ylim(-2.5, 2.5)
    ax.set_title(f'Degree {deg}  |  R²={r2:.3f}, adj-R²={adj_r2:.3f}')
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.legend(fontsize=9)

plt.suptitle('Polynomial Regression: Underfitting → Overfitting', fontsize=13)
plt.tight_layout()
plt.show()

print('Training R² by degree:')
for d, r2 in zip(degrees, r2_scores):
    bar = '#' * int(r2 * 40)
    print(f'  Degree {d:>2}: R²={r2:.4f}  |{bar}')

## 5. Logistic Regression (Binary Classification)

When the response is binary ($y \in \{0, 1\}$), linear regression is inappropriate. **Logistic regression** models the log-odds:

$$\log\frac{P(y=1 \mid x)}{1 - P(y=1 \mid x)} = \beta_0 + \beta_1 x$$

Transforming back via the **sigmoid function**:

$$P(y=1 \mid x) = \sigma(\beta_0 + \beta_1 x) = \frac{1}{1 + e^{-(\beta_0 + \beta_1 x)}}$$

Parameters are estimated by **maximum likelihood** (not OLS). The decision boundary is where $P = 0.5$, i.e., $\beta_0 + \beta_1 x = 0$.

**Interpretation of coefficients:** $e^{\beta_j}$ is the **odds ratio** — the factor by which the odds of $y=1$ multiply for a unit increase in $x_j$, holding other predictors constant.

In [ ]:
# Scenario: predict exam pass/fail from hours studied
n_logit = 100
hours   = rng.uniform(0, 10, n_logit)

# True logistic model: log-odds = -4 + 1.2 * hours
true_b0, true_b1 = -4.0, 1.2
log_odds = true_b0 + true_b1 * hours
prob_pass = 1 / (1 + np.exp(-log_odds))
passed = rng.binomial(1, prob_pass)                        # 0 = fail, 1 = pass

# Fit via scikit-learn (or manual gradient descent fallback)
if SKLEARN:
    X_logit = hours.reshape(-1, 1)
    clf = LogisticRegression(solver='lbfgs', C=1e6)        # large C → minimal regularisation
    clf.fit(X_logit, passed)
    b0_hat = clf.intercept_[0]
    b1_hat = clf.coef_[0, 0]
    print(f'Fitted (sklearn):   β₀={b0_hat:.4f}, β₁={b1_hat:.4f}')
else:
    # Manual gradient descent
    def sigmoid(z): return 1 / (1 + np.exp(-z))
    b0_hat, b1_hat = 0.0, 0.0
    lr = 0.05
    for _ in range(3000):
        p = sigmoid(b0_hat + b1_hat * hours)
        err = p - passed
        b0_hat -= lr * err.mean()
        b1_hat -= lr * (err * hours).mean()
    print(f'Fitted (grad desc): β₀={b0_hat:.4f}, β₁={b1_hat:.4f}')

print(f'True coefficients:  β₀={true_b0}, β₁={true_b1}')

# Decision boundary: b0 + b1*x = 0  →  x = -b0/b1
boundary = -b0_hat / b1_hat
print(f'Decision boundary   : {boundary:.2f} hours')
print(f'Odds ratio e^β₁     : {np.exp(b1_hat):.4f}')

# Visualise
x_range = np.linspace(0, 10, 300)
p_hat   = 1 / (1 + np.exp(-(b0_hat + b1_hat * x_range)))

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Left: sigmoid curve
ax = axes[0]
ax.scatter(hours[passed == 0], passed[passed == 0], color='tomato',    alpha=0.5, s=30,
           label='Fail (0)', zorder=3)
ax.scatter(hours[passed == 1], passed[passed == 1], color='steelblue', alpha=0.5, s=30,
           label='Pass (1)', zorder=3)
ax.plot(x_range, p_hat, 'k-', lw=2, label='Fitted P(pass)')
ax.axvline(boundary, color='green', ls='--', lw=1.5, label=f'Boundary = {boundary:.1f}h')
ax.axhline(0.5, color='gray', ls=':', lw=1)
ax.set_xlabel('Hours studied')
ax.set_ylabel('P(pass)')
ax.set_title('Logistic Regression: Sigmoid Curve')
ax.legend()

# Right: log-odds plot
ax = axes[1]
ax.plot(x_range, b0_hat + b1_hat * x_range, 'purple', lw=2, label='Log-odds')
ax.axhline(0, color='gray', ls='--')
ax.axvline(boundary, color='green', ls='--', lw=1.5, label=f'Boundary ({boundary:.1f}h)')
ax.fill_between(x_range, b0_hat + b1_hat * x_range, 0,
                where=(b0_hat + b1_hat * x_range > 0), alpha=0.15, color='steelblue', label='Predict pass')
ax.fill_between(x_range, b0_hat + b1_hat * x_range, 0,
                where=(b0_hat + b1_hat * x_range < 0), alpha=0.15, color='tomato', label='Predict fail')
ax.set_xlabel('Hours studied')
ax.set_ylabel('Log-odds')
ax.set_title('Log-odds and Decision Boundary')
ax.legend(fontsize=9)

plt.tight_layout()
plt.show()

## 6. Checking Regression Assumptions

OLS inference is valid only when the Gauss-Markov assumptions hold. Violations affect different inferential properties:

| Assumption | Consequence if violated | Diagnostic |
|------------|------------------------|------------|
| **Linearity** | Biased estimates | Residuals vs Fitted — look for curvature |
| **Independence** | Underestimated SEs | Durbin-Watson test (time-series) |
| **Homoscedasticity** | Inefficient estimates, invalid SEs | Scale-Location plot, Breusch-Pagan |
| **Normality of errors** | Invalid t/F intervals (small n) | Q-Q plot, Shapiro-Wilk test |
| **No perfect multicollinearity** | Singular (X'X), unreliable estimates | VIF > 10 is problematic |

Below we deliberately create two datasets — one that satisfies assumptions and one that violates **homoscedasticity** (heteroscedastic errors) — to compare the diagnostic plots.

In [ ]:
n_diag = 100
x_diag = rng.uniform(1, 10, n_diag)

# --- Dataset A: well-behaved (homoscedastic, normal errors)
y_good = 3 + 2 * x_diag + rng.normal(0, 1.5, n_diag)

# --- Dataset B: heteroscedastic (variance grows with x)
y_hetero = 3 + 2 * x_diag + rng.normal(0, 0.5 * x_diag, n_diag)

def fit_ols_1d(x, y):
    slope, intercept, *_ = stats.linregress(x, y)
    y_hat = intercept + slope * x
    resid = y - y_hat
    s = np.sqrt((resid**2).sum() / (len(x) - 2))
    return y_hat, resid, s

y_hat_good,   resid_good,   s_good   = fit_ols_1d(x_diag, y_good)
y_hat_hetero, resid_hetero, s_hetero = fit_ols_1d(x_diag, y_hetero)

def add_lowess_line(ax, x, y, frac=0.5, color='red'):
    """Simple moving average as a smoother (no statsmodels dependency)."""
    idx    = np.argsort(x)
    x_s, y_s = x[idx], y[idx]
    w = max(1, int(frac * len(x)))
    y_smooth = np.convolve(y_s, np.ones(w)/w, mode='same')
    ax.plot(x_s, y_smooth, color=color, lw=1.5, ls='--')

fig, axes = plt.subplots(2, 3, figsize=(13, 8))
titles_row = ['Homoscedastic (well-behaved)', 'Heteroscedastic (variance grows with x)']

for row, (x, y, y_hat, resid, s, label) in enumerate([
        (x_diag, y_good,   y_hat_good,   resid_good,   s_good,   'A: Well-behaved'),
        (x_diag, y_hetero, y_hat_hetero, resid_hetero, s_hetero, 'B: Heteroscedastic')]):

    std_resid = resid / s

    # (1) Residuals vs Fitted
    ax = axes[row, 0]
    ax.scatter(y_hat, resid, alpha=0.5, color='steelblue', s=20)
    ax.axhline(0, color='red', lw=1.5, ls='--')
    add_lowess_line(ax, y_hat, resid)
    ax.set_xlabel('Fitted'); ax.set_ylabel('Residuals')
    ax.set_title(f'{label}\nResiduals vs Fitted')

    # (2) Q-Q plot
    ax = axes[row, 1]
    (osm, osr), (sq, iq, rq) = stats.probplot(resid, dist='norm')
    ax.scatter(osm, osr, alpha=0.5, color='steelblue', s=20)
    ax.plot(osm, sq * np.array(osm) + iq, 'r-', lw=1.5)
    ax.set_xlabel('Theoretical quantiles'); ax.set_ylabel('Sample quantiles')
    ax.set_title('Normal Q-Q')

    # (3) Scale-Location
    ax = axes[row, 2]
    ax.scatter(y_hat, np.sqrt(np.abs(std_resid)), alpha=0.5, color='steelblue', s=20)
    add_lowess_line(ax, y_hat, np.sqrt(np.abs(std_resid)))
    ax.set_xlabel('Fitted'); ax.set_ylabel('√|Std residuals|')
    ax.set_title('Scale-Location')

    # Breusch-Pagan-like test: correlation between |residuals| and fitted values
    corr, bp_p = stats.pearsonr(y_hat, np.abs(resid))
    print(f'{label}: |residual| ~ fitted  r={corr:.3f}, p={bp_p:.4f}')

plt.suptitle('Regression Diagnostic Plots: Well-behaved vs Heteroscedastic', fontsize=13)
plt.tight_layout()
plt.show()

## Summary

### Regression Model Comparison

| Model | Response | Key idea | Fitting method | Interpretation |
|-------|----------|----------|----------------|----------------|
| Simple linear | Continuous | One predictor | OLS | β₁: change in ŷ per unit x |
| Multiple linear | Continuous | Multiple predictors | OLS | β_j: partial effect of x_j |
| Polynomial | Continuous | Non-linear with OLS | OLS in transformed space | Compare adj-R² across degrees |
| Logistic | Binary (0/1) | Log-odds linear in x | MLE (iterative) | e^β_j: odds ratio |

### Workflow for Any Regression Problem

1. **Explore** — scatter plots, correlation matrix, check for outliers
2. **Fit** — choose model family (linear / polynomial / logistic / etc.)
3. **Evaluate** — R², adjusted R², AIC/BIC, cross-validation MSE
4. **Diagnose** — residual plots, Q-Q plot, scale-location, leverage
5. **Iterate** — transform variables, add/remove predictors, consider regularisation (Ridge/Lasso)
6. **Report** — coefficient estimates, confidence intervals, effect sizes

### Extensions to Explore
- **Regularised regression**: Ridge (L2), Lasso (L1), ElasticNet — handle multicollinearity and feature selection
- **Generalised Linear Models (GLM)**: Poisson (count data), Negative Binomial, Gamma — when errors are non-normal
- **Mixed-effects models**: repeated measures and nested data structures
- **Robust regression**: Huber loss — reduces sensitivity to outliers
- **Non-parametric regression**: Kernel regression, GAM — flexible functional forms